In [1]:
from data_utils import extract_data, load_data, preprocess_data
import pandas as pd


In [2]:
data = load_data("data")

In [3]:
X_train, X_test, y_train, y_test = preprocess_data(
    data,
    add_post_text=True,
    include_pooled_comments=False,
    add_bertscore=False,
    n_negative_samples_per_link=1,
    add_title_embedding=True,
    add_body_embedding=True,
    title_embedding_dim=32,
    body_embedding_dim=32,
)

/home/leo/recsys/data_utils.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_features['body'] = posts_features['body'].map(clean_html)


Batches:   0%|          | 0/370 [00:00<?, ?it/s]

/home/leo/recsys/venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batches:   0%|          | 0/370 [00:00<?, ?it/s]

/home/leo/recsys/venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6391 entries, 3806 to 7779
Columns: 154 entries, id to comment_count_related
dtypes: float32(128), float64(6), int64(1), object(19)
memory usage: 4.4+ MB


In [5]:
X_train["linktypeid"].value_counts()

linktypeid
0    3407
1    2984
Name: count, dtype: int64

In [6]:
X_test["linktypeid"].value_counts()

linktypeid
0    865
1    733
Name: count, dtype: int64

In [5]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

# Define categorical features indices if you have any
categorical_features_indices = []  # e.g., [0, 1, 2] if your first three columns are categorical

# Initialize CatBoostClassifier
model = CatBoostClassifier(
    iterations=1000,  # Number of boosting iterations
    learning_rate=0.1,  # Learning rate
    depth=6,  # Depth of the tree
    cat_features=categorical_features_indices,  # Specify categorical features
    # text_features=["body", "comments_text", "comments_text_related", "body_related"],
    text_features=["body", "body_related", "title", "title_related"],
    verbose=100  # Print training progress every 100 iterations
)

# Train the model
model.fit(
    X_train.drop(columns=["postid", "relatedpostid", "id", "creationdate", "postid_related", "linktypeid"]),
    y_train,
    eval_set=(
        X_test.drop(columns=["postid", "relatedpostid", "id", "creationdate", "postid_related", "linktypeid"]),
        y_test
    ),
    plot=True
)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6642122	test: 0.6655945	best: 0.6655945 (0)	total: 94.9ms	remaining: 1m 34s
100:	learn: 0.2567135	test: 0.2993851	best: 0.2993851 (100)	total: 6.62s	remaining: 58.9s
200:	learn: 0.1665557	test: 0.2453283	best: 0.2453283 (200)	total: 13.1s	remaining: 52.1s
300:	learn: 0.1283085	test: 0.2346417	best: 0.2344523 (296)	total: 19.6s	remaining: 45.4s
400:	learn: 0.1004247	test: 0.2278117	best: 0.2278117 (400)	total: 26.1s	remaining: 38.9s
500:	learn: 0.0818841	test: 0.2242600	best: 0.2241052 (492)	total: 32.6s	remaining: 32.5s
600:	learn: 0.0673194	test: 0.2231829	best: 0.2231829 (600)	total: 39.1s	remaining: 26s
700:	learn: 0.0558529	test: 0.2210017	best: 0.2210017 (700)	total: 46s	remaining: 19.6s
800:	learn: 0.0480731	test: 0.2197535	best: 0.2196021 (772)	total: 52.5s	remaining: 13s
900:	learn: 0.0416278	test: 0.2196730	best: 0.2193379 (821)	total: 59s	remaining: 6.48s
999:	learn: 0.0364196	test: 0.2205989	best: 0.2193379 (821)	total: 1m 5s	remaining: 0us

bestTest = 0.21933785

In [10]:
# Make predictions
y_pred = model.predict(X_test.drop(columns=["postid", "relatedpostid", "id", "creationdate", "postid_related", "linktypeid"]))
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9174


In [11]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,body,7.595761
1,body_related,5.444646
2,vote_count_related,4.229183
3,title_related,2.746869
4,title,2.288367
...,...,...
141,body_embedding_9,0.055515
142,avg_comment_score_related,0.029222
143,comment_count_related,0.013717
144,commentcount_related,0.008301


# Calculate retrieve matrix

In [12]:
X_test_wo = X_test.drop(columns=["postid", "relatedpostid", "id", "creationdate", "postid_related", "linktypeid"])

In [13]:
len([col for col in X_test_wo.columns if col.endswith("_related")]), len(X_test_wo.columns) // 2

(73, 73)

In [55]:
posts = pd.concat([
    X_train[X_train["linktypeid"] == 1], 
    X_test[X_test["linktypeid"] == 1]
]).groupby("postid").first().reset_index(drop=False)
posts = posts.loc[:, ~posts.columns.duplicated()]
test_posts = posts[posts["postid"].isin(X_test["postid"])]

test_posts.drop(columns=["id", "creationdate", "linktypeid"], inplace=True)
test_posts.drop(columns=[col for col in posts.columns if col.endswith("_related")], inplace=True)

related_columns = posts.filter(like='_related').rename(columns=lambda x: x.replace('_related', ''))
posts.drop(columns=[col for col in posts.columns if col.endswith("_related")], inplace=True)
posts.drop(columns=['id', 'creationdate', 'relatedpostid', 'linktypeid'], inplace=True)
posts.reset_index(drop=True, inplace=True)
related_columns.reset_index(drop=False, inplace=True)
# print([ col for col in related_columns.columns if col not in posts.columns])
# print([ col for col in posts.columns if col not in related_columns.columns])
posts = pd.concat([posts, related_columns], axis=0, ignore_index=True)
posts = posts.groupby("postid").first().reset_index(drop=False)

print(set(test_posts.columns) - set(posts.columns))
print(set(posts.columns) - set(test_posts.columns))
len(test_posts.columns), len(posts.columns)

{'relatedpostid'}
{'index'}


/tmp/ipykernel_174139/28004364.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_posts.drop(columns=["id", "creationdate", "linktypeid"], inplace=True)
/tmp/ipykernel_174139/28004364.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_posts.drop(columns=[col for col in posts.columns if col.endswith("_related")], inplace=True)


(75, 75)

In [56]:
len(posts), len(test_posts)

(3754, 805)

In [57]:
train_columns = X_train.drop(columns=["postid", "relatedpostid", "id", "creationdate", "postid_related", "linktypeid"]).columns

In [61]:
from sklearn.metrics import ndcg_score
from tqdm.auto import tqdm

def compute_retrieve_metrics(model, test_posts, posts, ks=[5, 10, 30, 50, 100]):
    results = {k: {'ndcg_scores': [], 'recall_scores': []} for k in ks}

    for idx, test_post in tqdm(test_posts.iterrows()):
        # Concatenate test post features with all posts
        true_doc = test_post["relatedpostid"]
        test_post.drop(columns=["relatedpostid"], inplace=True)
        combined = pd.concat([test_post.to_frame().T] * len(posts), ignore_index=True)
        combined = pd.concat([combined.reset_index(drop=True), posts.add_suffix('_related')], axis=1)
        combined = combined[train_columns]
        
        # Compute model scores
        scores = model.predict_proba(combined)[:, 1]
        
        for k in ks:
            # Get top k posts
            top_k_indices = scores.argsort()[-k:][::-1]
            top_k_posts = posts.iloc[top_k_indices]
            
            # Compute recall@k
            recall_at_k = 1 if true_doc in top_k_posts["postid"].values else 0
            results[k]['recall_scores'].append(recall_at_k)
            
            # Compute ndcg@k
            true_relevance = [1 if post_id == true_doc else 0 for post_id in top_k_posts["postid"]]
            ndcg_at_k = ndcg_score([true_relevance], [scores[top_k_indices]])
            results[k]['ndcg_scores'].append(ndcg_at_k)
    
    avg_results = {}
    for k in ks:
        avg_recall_at_k = sum(results[k]['recall_scores']) / len(results[k]['recall_scores'])
        avg_ndcg_at_k = sum(results[k]['ndcg_scores']) / len(results[k]['ndcg_scores'])
        avg_results[k] = (avg_recall_at_k, avg_ndcg_at_k)
    
    return avg_results


In [64]:
avg_results = compute_retrieve_metrics(model, test_posts, posts)
for k in avg_results.keys():
    print(f"Average Recall@{k}: {avg_results[k][0]}")
    print(f"Average NDCG@{k}: {avg_results[k][1]}")

0it [00:00, ?it/s]

Average Recall@5: 0.24720496894409938
Average NDCG@5: 0.1611236824458043
Average Recall@10: 0.37267080745341613
Average NDCG@10: 0.20207455321651535
Average Recall@30: 0.5565217391304348
Average NDCG@30: 0.24589369414980294
Average Recall@50: 0.6409937888198758
Average NDCG@50: 0.2618682111361976
Average Recall@100: 0.7565217391304347
Average NDCG@100: 0.28063081048908284


In [65]:
from tqdm.auto import tqdm

def display_closest_documents(model, test_posts, posts, k=5):
    for idx, test_post in tqdm(test_posts.iterrows(), total=test_posts.shape[0]):
        # Prepare the test post for prediction
        test_post_features = test_post.drop(columns=["relatedpostid"]).to_frame().T
        combined = pd.concat([test_post_features] * len(posts), ignore_index=True)
        combined = pd.concat([combined.reset_index(drop=True), posts.add_suffix('_related')], axis=1)
        combined = combined[train_columns]
        
        # Compute model scores
        scores = model.predict_proba(combined)[:, 1]
        
        # Get top k posts
        top_k_indices = scores.argsort()[-k:][::-1]
        top_k_posts = posts.iloc[top_k_indices]
        
        # Display the results
        print(f"Test Post ID: {test_post['postid']}")
        print(f"Test Post Title: {test_post['title']}")
        print(f"Test Post Body: {test_post['body'][:100]}...")

        print(f"Top {k} closest documents:")
        for i, post in top_k_posts.iterrows():
            print(f"-"*100)
            print(f"Post ID: {post['postid']}")
            print(f"Title: {post['title']}")
            print(f"Body: {post['body'][:100]}...")
            print(f"-"*100)
        print("\n")

# Example usage
display_closest_documents(model, test_posts.iloc[:10], posts, k=5)

  0%|          | 0/10 [00:00<?, ?it/s]

Test Post ID: 10042
Test Post Title: How to make softer biscuits?
Test Post Body: Sorry to ask a rather non-specific question, but I am actually trying to formulate an example. Suppo...
Top 5 closest documents:
----------------------------------------------------------------------------------------------------
Post ID: 784
Title: Translating cooking terms between US / UK / AU / CA / NZ
Body: This post is an attempt to keep track of the terms that differ between dialects of English or exist ...
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Post ID: 14789
Title: Difference between Maida and All purpose flour
Body: I'm baking tonight and I'm out of All purpose flour. I'm lazy to run out to the grocery store. Can I...
----------------------------------------------------------------------------------------------------
-------------------